In [ ]:
import json
import glob
import os
from datetime import datetime

# Folder containing your JSON files
FOLDER = r"/Users/avduarte/Documents/GitHub/gym-dashboard.github.io/data"

# Define the replacements you want
REPLACEMENTS = {
    "Chest Press": "Bench Press",
    "Incline Chest Press": "Incline Bench Press",
}

def sorted_json_files(folder):
    """Find all .json files named DD-MM-YYYY.json and return them sorted by date."""
    files = glob.glob(os.path.join(folder, "*.json"))
    def parse_date(fname):
        basename = os.path.basename(fname)
        try:
            return datetime.strptime(basename, "%d-%m-%Y.json")
        except ValueError:
            return None
    dated = [(parse_date(f), f) for f in files]
    # Filter out any files that don't match the pattern
    dated = [df for df in dated if df[0] is not None]
    dated.sort(key=lambda x: x[0])
    return [f for (_, f) in dated]

def replace_exercises(obj):
    """
    Recursively walk through a loaded JSON object (dict/list)
    and replace any matching Exercise values.
    """
    if isinstance(obj, dict):
        for k, v in obj.items():
            if k == "Exercise" and isinstance(v, str) and v in REPLACEMENTS:
                obj[k] = REPLACEMENTS[v]
            else:
                replace_exercises(v)
    elif isinstance(obj, list):
        for item in obj:
            replace_exercises(item)

def process_all():
    files = sorted_json_files(FOLDER)
    for file_path in files:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        replace_exercises(data)

        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"Processed {os.path.basename(file_path)}")

if __name__ == "__main__":
    process_all()
